# Examining The Influence Of Square Footage On The Selling Price Of Houses In Vancouver #

## Introduction: ##
The Vancouver housing market is notorious for its skyrocketing prices-- making it the "second-most unaffordable [housing] market" after Hong Kong according to Kwan, Bloomberg News.

A newcomer to Vancouver looking for a home might wonder what "cheap" is when looking for homes. On the flip side, a seller might wonder whether they are pricing their home below, above, or at the market expectation in order to accelerate or deccelerate their time-to-sale.

(cite this later https://www.bnnbloomberg.ca/hong-kong-housing-ranked-world-s-least-affordable-for-9th-year-1.1201263)

The price of a house can depend on a multitude of factors, including location, size, the age of the home, and many others. Based on information about the property, predictions can be made about the selling price of a home compared to others in the area.

The goal of our project is to utilize regression analysis to determine the relationship between price and total square footage of houses in Vancouver. We will do this using a  [publicly available dataset from the website Kaggle](https://www.kaggle.com/datasets/darianghorbanian/vancouver-home-price-analysis-regression), which has price data, square footage, and other details about Vancouver houses from 2017-2020. 

Our Predictive Question is: “Does the square footage of a house in Vancouver have an impact on its selling price?”

# Methods & Results: #


## Reading data from the Web: ##